In [64]:
import pandas as pd

In [65]:
replacements = {'IT Audit / IT Compliance': 'Risk/Legal/Compliance',
               'IT': 'IT',
               'Engineering': 'Engineering',
               'Purchasing': 'Procurement',
               'Legal': 'Risk/Legal/Compliance',
               'Finance': 'Non-ICP',
               'Marketing': 'Non-ICP',
               'Sales': 'Non-ICP',
               'Unknown': 'Non-ICP',
               'Facilities': 'Non-ICP',
               'Human Resource': 'Non-ICP',
               'Management': 'Non-ICP',
               'Services': 'Non-ICP',
               'Operations': 'Non-ICP',
               'Administration': 'Non-ICP',
               'Corporate': 'Non-ICP',
               'Support': 'Non-ICP',
               'Education': 'Non-ICP',
               'Public Sector': 'Non-ICP',
               'Procurement': 'Procurement',
               'Medical': 'Non-ICP',
               'Other': 'Non-ICP',
               'IT Audit / IT Compliance': 'Risk/Legal/Compliance',
               'Information Security': 'IT',
               'IT - Security': 'IT',
               'Help Desk / Desktop Services': 'IT',
               'Information Technology': 'IT',
               'Infrastructure': 'Non-ICP',
               'Customer Service / Support': 'Non-ICP',
               'Emerging Technology / Innovation': 'IT'}

In [66]:
# Create dictionary for Job Role within IT function
replacements_role = {'Information Security': 'Information Security',
               'information security': 'Information Security',
               'Networking': 'Networking',
               'IT General': 'IT General',
               'None Technical': 'IT General',
               'Help Desk': 'IT General',
               'Governance Risk Compliance': 'IT General',
               'Program Management': 'IT General',
               'Data': 'IT General',
               'IT Facilities': 'IT General',
               'Operations': 'IT General',
               'Communications': 'IT General',
               'Integration': 'IT General',
               'Vendor Management': 'IT General',
               'Training': 'IT General',
               'Business Continuity': 'IT General',
               'Other': 'IT General',
               'Development': 'Development',
                'Security': 'Information Security',
                    'Business Systems': 'Systems'}

In [67]:
# Create dictionary for Job Level within all ICP functions
replacements_level = {'Manager': 'Manager',
                     'Contributor': 'Contributor',
                      'contributor': 'Contributor',
                     'Director': 'Director',
                     'C-Level': 'C-level',
                      'C-level': 'C-level',
                     'Executive': 'Executive',
                     'Unknown': pd.NA,
                      'Non-Manager': 'Contributor',
                      'VP-level': 'Executive',
                      'VP-Level': 'Executive',
                      'Decision maker': 'Manager',
                      'Team Lead': 'Manager',
                      'VP/Director': 'Director',
                      'Engineer/Admin': 'Contributor',
                      'CxO': 'C-level',
                      'VP': 'Executive',
                      'Director / C-Level': 'C-level',
                      'Individual Contributor': 'Contributor',
                      'Director Level': 'Director',
                      'contribtuor': 'Contributor',
                      'Management': 'Manager',
                      'Director of Enterprise Cloud Business': 'Director',
                      'Admin': 'Contributor'
                     }

In [68]:
def replace_role(row):
    if pd.isnull(row['Job Function']): 
        return 'N/A'
    elif 'IT' not in row['Job Function']:
        return 'N/A'
    else:
        return row['Job Role']

In [69]:
df = pd.read_csv('df_filtered.csv')
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df['Job Function'] = df['Job Function'].replace(replacements)
df['Job Function'] = df['Job Function'].fillna('Non-ICP')
df['Job Role'] = df.apply(replace_role, axis=1)
df['Job Level'] = df['Job Level'].replace(replacements_level)
df.loc[df['Job Function']=='Non-ICP', 'Job Level'] = pd.NA
df['Job Level'] = df['Job Level'].fillna(pd.NA)
df = df.dropna(subset=['Job Level'])



In [70]:
# Remove any stop words from the titles
from gensim.parsing.preprocessing import remove_stopwords
df['Title'] = df['Title'].apply(remove_stopwords)

In [71]:
import gensim 
#Some more preprocessing that will remove punctuations. 
tokenised_titles = df['Title'].apply(gensim.utils.simple_preprocess)

In [72]:
tokenised_titles.head()

0            [manager, cybersecurity]
1    [manager, information, security]
2         [user, experience, analyst]
3               [network, specialist]
4     [director, privacy, compliance]
Name: Title, dtype: object

In [73]:
#add the tokens as a column in df
df.insert(1, 'Title_Tokens', tokenised_titles)
df.head()

,Title,Title_Tokens,Job Role,Job Function,Job Level
0,manager-cybersecurity,"[manager, cybersecurity]",Information Security,IT,Manager
1,"manager, information security","[manager, information, security]",Information Security,IT,Manager
2,user experience analyst,"[user, experience, analyst]",N/A,Engineering,Contributor
3,network specialist,"[network, specialist]",Networking,IT,Contributor
4,director privacy compliance,"[director, privacy, compliance]",Information Security,IT,Director


In [74]:
#initialize and train model to create word vectors
model = gensim.models.Word2Vec(window=2,min_count=1, workers=4)
model.build_vocab(df['Title_Tokens'])#this is a required step before training the model 
model.train(df['Title_Tokens'], total_examples=model.corpus_count, epochs=model.epochs) #default vector_size = 100


(3573814, 8613425)

In [76]:
model.wv.most_similar("manager")

[('director', 0.7643967866897583),
 ('leader', 0.7204034924507141),
 ('manger', 0.6554084420204163),
 ('specialist', 0.6410418748855591),
 ('supervisor', 0.640966534614563),
 ('mgr', 0.62857586145401),
 ('coordinator', 0.6219028830528259),
 ('lead', 0.620930552482605),
 ('analyst', 0.6153537034988403),
 ('manage', 0.611866295337677)]

In [77]:
model.wv.most_similar("analyst")

[('specialist', 0.8318034410476685),
 ('analysts', 0.7098676562309265),
 ('technician', 0.6657198071479797),
 ('ecss', 0.6575592756271362),
 ('manger', 0.6439298391342163),
 ('mgr', 0.6319141387939453),
 ('engineer', 0.6283688545227051),
 ('manager', 0.6153537631034851),
 ('sbi', 0.611548125743866),
 ('anaylst', 0.6086878180503845)]

In [78]:
model.wv.most_similar("network")

[('networks', 0.6262040138244629),
 ('noc', 0.6151784062385559),
 ('video', 0.6038103699684143),
 ('acoms', 0.5920001864433289),
 ('ip', 0.5913484692573547),
 ('carrier', 0.5748663544654846),
 ('flight', 0.5701092481613159),
 ('networking', 0.5645323395729065),
 ('mac', 0.5577559471130371),
 ('lan', 0.5525267720222473)]

In [79]:
import numpy as np

#below function creates a vector for a title by adding the vector of each word in it
def get_title_vec(title_tokens):
    if len(title_tokens) == 0:
        return np.zeros(100)

    # create a list of vectors of all the tokens in a title
    vectors = [ model.wv[token] for token in title_tokens]
    #sum all the vectors in the list
    return np.sum(vectors, axis=0)

title_vecs = df['Title_Tokens'].map(get_title_vec)
#Add title vecs as a column in df 
df.insert(2, 'Title_Vec', title_vecs)
df.head()



,Title,Title_Tokens,Title_Vec,Job Role,Job Function,Job Level
0,manager-cybersecurity,"[manager, cybersecurity]","[-0.7246598, 1.2171857, -0.25482756, -0.192139...",Information Security,IT,Manager
1,"manager, information security","[manager, information, security]","[-1.2036567, 1.4740264, -0.274158, -1.0400465,...",Information Security,IT,Manager
2,user experience analyst,"[user, experience, analyst]","[-0.22868088, 1.397039, 0.2054277, -2.310388, ...",N/A,Engineering,Contributor
3,network specialist,"[network, specialist]","[-1.4234097, 1.3662905, 0.06710291, -0.5908642...",Networking,IT,Contributor
4,director privacy compliance,"[director, privacy, compliance]","[-0.52955604, 0.8572507, -1.4817626, -0.687059...",Information Security,IT,Director


In [80]:
#create training and validation sets for FUNCTION, ROLE, LEVEL for training/crossvalidating classification models 
from sklearn.model_selection import train_test_split

trainX,validX,trainy,validy = train_test_split(df['Title_Vec'], df['Job Function'],train_size=0.80,random_state=123)

it_function = df[df['Job Function'].str.contains('IT')]
it_function['Job Role'] = it_function['Job Role'].replace(replacements_role)
it_function['Job Role'] = it_function['Job Role'].fillna('IT General')
trainX_role, validX_role, trainy_role, validy_role = train_test_split(it_function['Title_Vec'], it_function['Job Role'], train_size=0.80, random_state=123)
trainX_level, validX_level, trainy_level, validy_level = train_test_split(it_function['Title_Vec'], it_function['Job Level'], train_size=0.80, random_state=123)


c:\Users\rajat\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\Users\rajat\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [81]:
# Start with Bayes Naive Classifier to determine Job Function 
from sklearn.naive_bayes import GaussianNB as NBC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV

nbc = NBC()
# Parameters to tune for NBC
nbc_params = {'var_smoothing': [0.1, 0.5, 1.0, 2.0]}

rand_search_nbc = RandomizedSearchCV(NBC(),nbc_params, # positional arguments, model and parameter grid
                                    n_iter=10,
                                    scoring='f1_macro', 
                                    cv=5,
                                    random_state=123,
                                    n_jobs=-1)

rand_search_nbc.fit(trainX.tolist(),trainy)
print(f"Classification Report with hold-out sample for best fit of NBC model:...\n")
print(classification_report(validy,rand_search_nbc.predict(validX.tolist())))
print("------------------------------------------------------")
print("Best estmator for NBC model for JOB FUNCTION: {}".format(rand_search_nbc.best_estimator_), )

c:\Users\rajat\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Classification Report with hold-out sample for best fit of NBC model:...

                       precision    recall  f1-score   support

          Engineering       0.24      0.85      0.37      8811
                   IT       0.97      0.72      0.83     87382
          Procurement       1.00      0.00      0.01       239
Risk/Legal/Compliance       0.04      0.18      0.06       156

             accuracy                           0.73     96588
            macro avg       0.56      0.44      0.32     96588
         weighted avg       0.91      0.73      0.78     96588

------------------------------------------------------
Best estmator for NBC model for JOB FUNCTION: GaussianNB(priors=None, var_smoothing=2.0)


In [82]:
rand_search_nbc_role = RandomizedSearchCV(NBC(),nbc_params, # positional arguments, model and parameter grid
                                    n_iter=10,
                                    scoring='f1_macro', 
                                    cv=5,
                                    random_state=123,
                                    n_jobs=-1)
    
rand_search_nbc_role.fit(trainX_role.tolist(),trainy_role)
print(f"  Report with hold-out sample for best fit of NBC model:...\n")
print(classification_report(validy_role,rand_search_nbc_role.predict(validX_role.tolist())))
print("------------------------------------------------------")
print("Best estmator for NBC model for JOB ROLE: {}".format(rand_search_nbc_role.best_estimator_), )

c:\Users\rajat\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


  Report with hold-out sample for best fit of NBC model:...

                      precision    recall  f1-score   support

         Development       0.19      0.38      0.25      1982
          IT General       0.38      0.17      0.23     14579
Information Security       0.76      0.46      0.57     39367
          Networking       0.49      0.83      0.61     29474
             Systems       0.24      0.38      0.29      1984

            accuracy                           0.53     87386
           macro avg       0.41      0.44      0.39     87386
        weighted avg       0.58      0.53      0.52     87386

------------------------------------------------------
Best estmator for NBC model for JOB ROLE: GaussianNB(priors=None, var_smoothing=0.1)


In [83]:
rand_search_nbc_level = RandomizedSearchCV(NBC(),nbc_params, # positional arguments, model and parameter grid
                                    n_iter=10,
                                    scoring='f1_macro', 
                                    cv=5,
                                    random_state=123,
                                    n_jobs=-1)
    
rand_search_nbc_level.fit(trainX_level.tolist(),trainy_level)
print(f"  Report with hold-out sample for best fit of NBC model:...\n")
print(classification_report(validy_level,rand_search_nbc_level.predict(validX_level.tolist())))
print("------------------------------------------------------")
print("Best estmator for NBC model for JOB LEVEL: {}".format(rand_search_nbc_level.best_estimator_), )

c:\Users\rajat\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


  Report with hold-out sample for best fit of NBC model:...

              precision    recall  f1-score   support

     C-level       0.72      0.81      0.76      9497
 Contributor       0.80      0.87      0.83     28226
    Director       0.78      0.86      0.82     18855
   Executive       0.56      0.42      0.48     11641
     Manager       0.87      0.75      0.80     19167

    accuracy                           0.77     87386
   macro avg       0.75      0.74      0.74     87386
weighted avg       0.77      0.77      0.77     87386

------------------------------------------------------
Best estmator for NBC model for JOB LEVEL: GaussianNB(priors=None, var_smoothing=0.1)


In [84]:
from sklearn.tree import DecisionTreeClassifier as DTC

dt_params = {'criterion':['gini'],
             'max_depth':[10, 20, 50],
             'min_samples_split':[2,5,10], # required to split
             'max_features':['sqrt','log2'],
             'random_state':[123]
            }

rand_search_DTC = RandomizedSearchCV(DTC(),dt_params, # positional arguments, model and parameter grid
                                    n_iter=10,
                                    scoring='f1_macro', 
                                    cv=5,
                                    random_state=123,
                                    n_jobs=-1)
    
rand_search_DTC.fit(trainX.tolist(),trainy)
print(f"Classification Report with hold-out sample for best fit of DTC model:...\n")
print(classification_report(validy,rand_search_DTC.predict(validX.tolist())))
print("------------------------------------------------------")
print("Best estmator for DTC model for JOB FUNCTION: {}".format( rand_search_DTC.best_estimator_), )
    

c:\Users\rajat\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Classification Report with hold-out sample for best fit of DTC model:...

                       precision    recall  f1-score   support

          Engineering       0.88      0.93      0.90      8811
                   IT       0.99      0.99      0.99     87382
          Procurement       0.69      0.72      0.71       239
Risk/Legal/Compliance       0.79      0.69      0.74       156

             accuracy                           0.98     96588
            macro avg       0.84      0.83      0.83     96588
         weighted avg       0.98      0.98      0.98     96588

------------------------------------------------------
Best estmator for DTC model for JOB FUNCTION: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=50, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       mi

In [85]:
rand_search_dtc_role = RandomizedSearchCV(DTC(),dt_params, # positional arguments, model and parameter grid
                                    n_iter=10,
                                    scoring='f1_macro', 
                                    cv=5,
                                    random_state=123,
                                    n_jobs=-1)
    
rand_search_dtc_role.fit(trainX_role.tolist(),trainy_role)
print(f"  Report with hold-out sample for best fit of DTC model:...\n")
print(classification_report(validy_role,rand_search_dtc_role.predict(validX_role.tolist())))
print("------------------------------------------------------")
print("Best estmator for DTC model for JOB ROLE: {}".format(rand_search_dtc_role.best_estimator_), )

  Report with hold-out sample for best fit of DTC model:...

                      precision    recall  f1-score   support

         Development       0.71      0.70      0.70      1982
          IT General       0.79      0.61      0.69     14579
Information Security       0.89      0.90      0.89     39367
          Networking       0.85      0.94      0.89     29474
             Systems       0.75      0.66      0.70      1984

            accuracy                           0.85     87386
           macro avg       0.80      0.76      0.78     87386
        weighted avg       0.85      0.85      0.85     87386

------------------------------------------------------
Best estmator for DTC model for JOB ROLE: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=50, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_sampl

In [86]:
rand_search_dtc_level = RandomizedSearchCV(DTC(),dt_params, # positional arguments, model and parameter grid
                                    n_iter=10,
                                    scoring='f1_macro', 
                                    cv=5,
                                    random_state=123,
                                    n_jobs=-1)
    
rand_search_dtc_level.fit(trainX_level.tolist(),trainy_level)
print(f"  Report with hold-out sample for best fit of DTC model:...\n")
print(classification_report(validy_level,rand_search_dtc_level.predict(validX_level.tolist())))
print("------------------------------------------------------")
print("Best estmator for DTC model for JOB LEVEL: {}".format(rand_search_dtc_level.best_estimator_), )

c:\Users\rajat\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


  Report with hold-out sample for best fit of DTC model:...

              precision    recall  f1-score   support

     C-level       0.93      0.93      0.93      9497
 Contributor       0.95      0.95      0.95     28226
    Director       0.95      0.95      0.95     18855
   Executive       0.92      0.92      0.92     11641
     Manager       0.94      0.94      0.94     19167

    accuracy                           0.94     87386
   macro avg       0.94      0.94      0.94     87386
weighted avg       0.94      0.94      0.94     87386

------------------------------------------------------
Best estmator for DTC model for JOB LEVEL: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=50, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presor

In [87]:
rf_params = {'n_estimators':[10,25],
             'criterion':['gini'],
             'max_depth':[10, 20, 50],
             'min_samples_split':[2,5,10], # required to split
             'max_features':['sqrt','log2'],
             'max_samples':[0.10,0.50],
             'random_state':[123]
            }

In [88]:
from sklearn.ensemble import RandomForestClassifier as RFC

rand_search_rfc_function = RandomizedSearchCV(RFC(),rf_params, # positional arguments, model and parameter grid
                                    n_iter=10,
                                    scoring='f1_macro', 
                                    cv=5,
                                    random_state=123,
                                    n_jobs=-1)
    
rand_search_rfc_function.fit(trainX.tolist(),trainy)
print(f"Classification Report with hold-out sample for best fit of rfc model:...\n")
print(classification_report(validy,rand_search_rfc_function.predict(validX.tolist())))
print("------------------------------------------------------")
print("Best estmator for RFC model for JOB FUNCTION: {}".format(rand_search_rfc_function.best_estimator_), )

c:\Users\rajat\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Classification Report with hold-out sample for best fit of rfc model:...

                       precision    recall  f1-score   support

          Engineering       0.92      0.91      0.92      8811
                   IT       0.99      0.99      0.99     87382
          Procurement       0.83      0.60      0.69       239
Risk/Legal/Compliance       1.00      0.60      0.75       156

             accuracy                           0.98     96588
            macro avg       0.93      0.78      0.84     96588
         weighted avg       0.98      0.98      0.98     96588

------------------------------------------------------
Best estmator for RFC model for JOB FUNCTION: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=0.5,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_sample

In [89]:
rand_search_rfc_role = RandomizedSearchCV(RFC(),rf_params, # positional arguments, model and parameter grid
                                    n_iter=10,
                                    scoring='f1_macro', 
                                    cv=5,
                                    random_state=123,
                                    n_jobs=-1)
    
rand_search_rfc_role.fit(trainX_role.tolist(),trainy_role)
print(f"  Report with hold-out sample for best fit of RFC model:...\n")
print(classification_report(validy_role,rand_search_rfc_role.predict(validX_role.tolist())))
print("------------------------------------------------------")
print("Best estmator for RFC model for JOB ROLE: {}".format(rand_search_rfc_role.best_estimator_), )

c:\Users\rajat\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


  Report with hold-out sample for best fit of RFC model:...

                      precision    recall  f1-score   support

         Development       0.81      0.67      0.73      1982
          IT General       0.81      0.60      0.69     14579
Information Security       0.88      0.91      0.89     39367
          Networking       0.85      0.96      0.90     29474
             Systems       0.85      0.63      0.72      1984

            accuracy                           0.86     87386
           macro avg       0.84      0.75      0.79     87386
        weighted avg       0.86      0.86      0.86     87386

------------------------------------------------------
Best estmator for RFC model for JOB ROLE: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=0.5,
                       min_impurity_decrease=0.0, min_impurity_split=No

In [90]:

rand_search_rfc_level = RandomizedSearchCV(RFC(),rf_params, # positional arguments, model and parameter grid
                                    n_iter=10,
                                    scoring='f1_macro', 
                                    cv=5,
                                    random_state=123,
                                    n_jobs=-1)
    
rand_search_rfc_level.fit(trainX_level.tolist(),trainy_level)
print(f"  Report with hold-out sample for best fit of RFC model:...\n")
print(classification_report(validy_level,rand_search_rfc_level.predict(validX_level.tolist())))
print("------------------------------------------------------")
print("Best estmator for RFC model for JOB LEVEL: {}".format(rand_search_rfc_level.best_estimator_), )

c:\Users\rajat\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


  Report with hold-out sample for best fit of RFC model:...

              precision    recall  f1-score   support

     C-level       0.94      0.93      0.94      9497
 Contributor       0.96      0.96      0.96     28226
    Director       0.96      0.97      0.96     18855
   Executive       0.94      0.93      0.94     11641
     Manager       0.95      0.96      0.96     19167

    accuracy                           0.95     87386
   macro avg       0.95      0.95      0.95     87386
weighted avg       0.95      0.95      0.95     87386

------------------------------------------------------
Best estmator for RFC model for JOB LEVEL: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features='log2',
                       max_leaf_nodes=None, max_samples=0.5,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
  